In [1]:
from torchsummary import summary

In [2]:
import os
import os.path as osp
import pprint
import random

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import skimage.io
import skimage.transform
import torch
import yaml
from docopt import docopt

import lcnn
from lcnn.config import C, M
from lcnn.models.line_vectorizer import LineVectorizer
from lcnn.models.multitask_learner import MultitaskHead, MultitaskLearner
from lcnn.postprocess import postprocess
from lcnn.utils import recursive_to

In [3]:
# args = docopt(__doc__)
config_file = "config/wireframe.yaml"
C.update(C.from_yaml(filename=config_file))
M.update(C.model)
pprint.pprint(C, indent=4)

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

device_name = "cpu"
# os.environ["CUDA_VISIBLE_DEVICES"] = args["--devices"]
if torch.cuda.is_available():
    device_name = "cuda"
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed(0)
    print("Let's use", torch.cuda.device_count(), "GPU(s)!")
else:
    print("CUDA is not available")
device = torch.device(device_name)
checkpoint = torch.load("190418-201834-f8934c6-lr4d10-312k.pth.tar", 
                        map_location=device)

{   'io': {   'datadir': 'data/wireframe/',
              'logdir': 'logs/',
              'num_workers': 4,
              'resume_from': None,
              'tensorboard_port': 0,
              'validation_interval': 24000},
    'model': {   'backbone': 'stacked_hourglass',
                 'batch_size': 6,
                 'batch_size_eval': 2,
                 'depth': 4,
                 'dim_fc': 1024,
                 'dim_loi': 128,
                 'eval_junc_thres': 0.008,
                 'head_size': <BoxList: [[2], [1], [2]]>,
                 'image': {   'mean': <BoxList: [109.73, 103.832, 98.681]>,
                              'stddev': <BoxList: [22.275, 22.124, 23.229]>},
                 'loss_weight': {   'jmap': 8.0,
                                    'joff': 0.25,
                                    'lmap': 0.5,
                                    'lneg': 1,
                                    'lpos': 1},
                 'n_dyn_junc': 300,
                 'n_dy

In [4]:
# loading model
model = lcnn.models.hg(
    depth=M.depth,
    head=lambda c_in, c_out: MultitaskHead(c_in, c_out),
    num_stacks=M.num_stacks,
    num_blocks=M.num_blocks,
    num_classes=sum(sum(M.head_size, [])),
)
model = MultitaskLearner(model)
model = LineVectorizer(model)
model.load_state_dict(checkpoint["model_state_dict"])
model = model.to(device)
model.eval()

LineVectorizer(
  (backbone): MultitaskLearner(
    (backbone): HourglassNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (layer1): Sequential(
        (0): Bottleneck2D(
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
     

In [10]:
# Grabbing random image
im = skimage.io.imread("/home/fcr/Pictures/photo_stuff.JPG")
if im.ndim == 2:
    im = np.repeat(im[:, :, None], 3, 2)
im = im[:, :, :3]
im_resized = skimage.transform.resize(im, (512, 512)) * 255
image = (im_resized - M.image.mean) / M.image.stddev
image = torch.from_numpy(np.rollaxis(image, 2)[None].copy()).float()
image.shape

torch.Size([1, 3, 512, 512])

In [12]:
input_dict = {
    "image": image.to(device),
    "meta": [
        {
            "junc": torch.zeros(1, 2).to(device),
            "jtyp": torch.zeros(1, dtype=torch.uint8).to(device),
            "Lpos": torch.zeros(2, 2, dtype=torch.uint8).to(device),
            "Lneg": torch.zeros(2, 2, dtype=torch.uint8).to(device),
        }
    ],
    "target": {
        "jmap": torch.zeros([1, 1, 128, 128]).to(device),
        "joff": torch.zeros([1, 1, 2, 128, 128]).to(device),
    },
    "mode": "testing",
}
summary(model, input_size=(3, 512, 512))

IndexError: too many indices for tensor of dimension 4

In [5]:
with torch.no_grad():
    input_dict = {
        "image": image.to(device),
        "meta": [
            {
                "junc": torch.zeros(1, 2).to(device),
                "jtyp": torch.zeros(1, dtype=torch.uint8).to(device),
                "Lpos": torch.zeros(2, 2, dtype=torch.uint8).to(device),
                "Lneg": torch.zeros(2, 2, dtype=torch.uint8).to(device),
            }
        ],
        "target": {
            "jmap": torch.zeros([1, 1, 128, 128]).to(device),
            "joff": torch.zeros([1, 1, 2, 128, 128]).to(device),
        },
        "mode": "testing",
    }
    model_out = model(input_dict)

In [7]:
model_out

{'feature': tensor([[[[0.7589, 0.8255, 0.4786,  ..., 0.5998, 1.0855, 1.0054],
           [0.9022, 0.6508, 0.3571,  ..., 0.5829, 0.7833, 0.9299],
           [0.7404, 0.5643, 0.3696,  ..., 0.4243, 0.5113, 0.7684],
           ...,
           [1.0381, 0.8115, 0.6234,  ..., 0.3036, 0.6313, 0.8480],
           [1.0483, 0.7447, 0.6298,  ..., 0.7488, 0.6024, 0.7668],
           [0.9986, 0.9757, 0.9153,  ..., 1.0050, 0.6442, 0.6412]],
 
          [[0.0287, 0.1454, 0.0619,  ..., 0.1879, 0.2072, 0.0857],
           [0.1558, 0.1764, 0.1529,  ..., 0.2814, 0.2966, 0.2091],
           [0.1383, 0.2292, 0.1491,  ..., 0.2525, 0.2828, 0.1659],
           ...,
           [0.1509, 0.2563, 0.1932,  ..., 0.2135, 0.2475, 0.1456],
           [0.1828, 0.2519, 0.2320,  ..., 0.2031, 0.2349, 0.1565],
           [0.1055, 0.2078, 0.1860,  ..., 0.1635, 0.1724, 0.0916]],
 
          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0069,  ..., 0.0053, 0.0000, 0.0000],
           [0

In [6]:
# Export the model
torch.onnx.export(model,                     # model being run
                  input_dict,                # model input (or a tuple for multiple inputs)
                  "lcnn.onnx",               # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['image'],   # the model's input names
                  output_names = ['jmap', 'joff'], # the model's output names
)

/home/fcr/research/lcnn/lcnn/models/line_vectorizer.py:163: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  N = len(junc)
/home/fcr/research/lcnn/lcnn/models/line_vectorizer.py:165: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  K = min(int((jmap > M.eval_junc_thres).float().sum().item()), max_K)
/home/fcr/research/lcnn/lcnn/models/line_vectorizer.py:165: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This 

KeyError: 'argsort'